<a href="https://colab.research.google.com/github/w86w86/econ8310-assignment1/blob/main/assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 1 - Forecast the # of taxi trips requested during each hour in a week

##Execution

###Upload the data and Format into Prophet column's name

In [21]:
file = "https://github.com/dustywhite7/econ8310-assignment1/raw/main/assignment_data_train.csv"
#Import statements
import pandas as pd
import numpy as np
from prophet import Prophet

# Import the dataset of #of taxi trips each hour in New York City,
trip = pd.read_csv(file)

# Keep only the dates and the y value
trip = trip[['Timestamp','trips']]
print('->trip.Timestamp (before) dtype: ', trip.Timestamp.dtype)

# Format the date (Timestamp
trip.Timestamp = pd.to_datetime(trip.Timestamp)
print('->trip.Timestamp (after) dtype: ', trip.Timestamp.dtype)

# Prophet format using column ds and y
trip=trip.rename(columns={'Timestamp':'ds','trips':'y'})
trip.head()

->trip.Timestamp (before) dtype:  object
->trip.Timestamp (after) dtype:  datetime64[ns]


,ds,y
0,2018-01-01 00:00:00,16714
1,2018-01-01 01:00:00,19041
2,2018-01-01 02:00:00,16590
3,2018-01-01 03:00:00,12626
4,2018-01-01 04:00:00,8739


###Graphic Representation of all trips (hourly)

In [22]:
import plotly.express as px
fig = px.line(trip, x='ds', y='y')
fig.show()

###Initialize GAM/Prophet instance and fit to data / modelFit

In [23]:
%%time
model = Prophet()
modelFit = model.fit(trip)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpqlols3i4/uzav2n41.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpqlols3i4/bf0va_w8.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=17388', 'data', 'file=/tmp/tmpqlols3i4/uzav2n41.json', 'init=/tmp/tmpqlols3i4/bf0va_w8.json', 'output', 'file=/tmp/tmpqlols3i4/prophet_modelzqgi14r5/prophet_model-20250917063700.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
06:37:00 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
06:37:01 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


CPU times: user 392 ms, sys: 18.4 ms, total: 410 ms
Wall time: 1.84 s


###Create timeline 31 days

In [24]:
future = modelFit.make_future_dataframe(periods=744, freq='h', include_history=False)
print('future.shape: ', future.shape)
future.tail()

future.shape:  (744, 1)


,ds
739,2019-01-31 19:00:00
740,2019-01-31 20:00:00
741,2019-01-31 21:00:00
742,2019-01-31 22:00:00
743,2019-01-31 23:00:00


In [25]:
forecast = modelFit.predict(future)
#forecast

In [26]:
pred = forecast.yhat.tolist()
print(pred)

[7618.173261325992, 5137.799878266395, 2657.221235155728, 862.5983133588088, 622.0615299630026, 2323.1878633341084, 5432.466880498393, 8730.090174217145, 11086.572933106278, 12153.355469466596, 12411.553810279162, 12598.368131877553, 13067.043165176627, 13636.835141803982, 13979.425007708007, 14078.101012276053, 14273.787061355999, 14869.839151168431, 15727.331478061968, 16284.124028255581, 15989.554577890434, 14724.26516661284, 12803.310182515459, 10602.766349385412, 8241.461710554822, 5678.28943734351, 3119.164381804826, 1250.944139435669, 942.3756307830063, 2581.566879232396, 5635.4487897118115, 8884.561474544678, 11199.67387558064, 12232.38451609591, 12463.87248486089, 12631.307413222587, 13087.812539753575, 13652.435385983572, 13996.565710584442, 14103.123778459418, 14312.595249972697, 14927.83491515394, 15809.360765763722, 16394.43010005224, 16131.74047558343, 14901.265453734191, 13017.37214997499, 10855.440757335713, 8533.603013410435, 6010.065314002368, 3490.0742780399423, 1659

###Dealing with the negative values

In [27]:
negative_pred = [(ind, val) \
                 for ind, val in enumerate(pred) if val<0]
print ('Nb of negative values:', len(negative_pred))
#negative_pred

Nb of negative values: 14


####Replace the negative value with min of trip in an hour from our data

In [28]:
pred = [max(min(trip.y), val) for val in pred]
print(pred)

[7618.173261325992, 5137.799878266395, 2657.221235155728, 862.5983133588088, 622.0615299630026, 2323.1878633341084, 5432.466880498393, 8730.090174217145, 11086.572933106278, 12153.355469466596, 12411.553810279162, 12598.368131877553, 13067.043165176627, 13636.835141803982, 13979.425007708007, 14078.101012276053, 14273.787061355999, 14869.839151168431, 15727.331478061968, 16284.124028255581, 15989.554577890434, 14724.26516661284, 12803.310182515459, 10602.766349385412, 8241.461710554822, 5678.28943734351, 3119.164381804826, 1250.944139435669, 942.3756307830063, 2581.566879232396, 5635.4487897118115, 8884.561474544678, 11199.67387558064, 12232.38451609591, 12463.87248486089, 12631.307413222587, 13087.812539753575, 13652.435385983572, 13996.565710584442, 14103.123778459418, 14312.595249972697, 14927.83491515394, 15809.360765763722, 16394.43010005224, 16131.74047558343, 14901.265453734191, 13017.37214997499, 10855.440757335713, 8533.603013410435, 6010.065314002368, 3490.0742780399423, 1659

##TEST OF MY CODE

###Imports

In [ ]:
import statsmodels.tsa.holtwinters as hw
from statsmodels.tsa.statespace.varmax import VARMAXResultsWrapper
from statsmodels.tsa.vector_ar.var_model import VARResultsWrapper

In [ ]:
# testValidModel
import prophet
print("isinstance(model, prophet.forecaster.Prophet):   ", isinstance(modelFit, prophet.forecaster.Prophet))

modelFit.history

isinstance(model, prophet.forecaster.Prophet):    True


,ds,y,floor,t,y_scaled
0,2018-01-01 00:00:00,16714,0.0,0.000000,0.699770
1,2018-01-01 01:00:00,19041,0.0,0.000114,0.797195
2,2018-01-01 02:00:00,16590,0.0,0.000228,0.694578
3,2018-01-01 03:00:00,12626,0.0,0.000343,0.528616
4,2018-01-01 04:00:00,8739,0.0,0.000457,0.365878
...,...,...,...,...,...
8755,2018-12-31 19:00:00,14876,0.0,0.999543,0.622818
8756,2018-12-31 20:00:00,14434,0.0,0.999657,0.604312
8757,2018-12-31 21:00:00,14116,0.0,0.999772,0.590999
8758,2018-12-31 22:00:00,10729,0.0,0.999886,0.449194


In [ ]:
# testFittedModel.py
print( 'isinstance(modelFit.history, pd.DataFrame): ', isinstance(modelFit.history, pd.DataFrame) )

isinstance(modelFit.history, pd.DataFrame):  True


In [ ]:
# testAccuracy1.py,  testAccuracy2.py,  testAccuracy3.py,
# RMSE
dataTest=trip.y
rmse = sum([(np.squeeze(pred)[i]-dataTest[i])**2 for i in range(len(np.squeeze(pred)))])
rmse = np.sqrt(rmse)*1/744

print ('rmse: ', rmse)
if rmse<171:
  print ('rmse<171')
else:
  print("Your forecasts have an RMSE above 220 or 185 or 171")

rmse:  143.07372275903958
rmse<171


#Tests From Dr White

###imports and Run Function Check

In [29]:
#!pip install pygam

In [30]:
import unittest
import patsy as pt
from pygam import LinearGAM, f, s, l
import numpy as np
import pandas as pd
import statsmodels as statsmodels
import pygam
from statsmodels.tsa.holtwinters import ExponentialSmoothing as ES
import prophet
import statsmodels.tsa.holtwinters as hw

# Import your code from parent directory
import os
import sys
import inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

#from assignment1 import model, modelFit, pred

In [42]:
bool(modelFit.history)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [31]:
# Run the checks
def checkNumbers(series):
  for i in series:
    if not isinstance(i, (float, int)):
      return False
  return True


def test_valid_model():
    return ( isinstance(model, prophet.forecaster.Prophet), "Make sure that you are using a model\ncovered in class in the variable 'model'.")

def test_fitted_model():
  if hasattr(modelFit, 'history'):
      return (bool(modelFit.history))
  else:
    print("Make sure that you store your fitted model in the variable 'modelFit'.")
    return (False)

def test_valid_pred():
  return (len(np.squeeze(pred))==744 and checkNumbers(np.squeeze(pred))), "Make sure your prediction consists of integers\nor floating point numbers, and is a list or array of 744\nfuture predictions!"

def test_accuracy_level_1():
  dataTest = pd.read_csv(currentdir + "/testData.csv")['trips']
  rmse = sum([(np.squeeze(pred)[i]-dataTest[i])**2 for i in range(len(np.squeeze(pred)))])
  rmse = np.sqrt(rmse)*1/744

  return (rmse<220)

def test_accuracy_level_2():
  dataTest = pd.read_csv(currentdir + "/testData.csv")['trips']
  rmse = sum([(np.squeeze(pred)[i]-dataTest[i])**2 for i in range(len(np.squeeze(pred)))])
  rmse = np.sqrt(rmse)*1/744

  return (rmse<185)

def test_accuracy_level_3():
  dataTest = pd.read_csv(currentdir + "/testData.csv")['trips']
  rmse = sum([(np.squeeze(pred)[i]-dataTest[i])**2 for i in range(len(np.squeeze(pred)))])
  rmse = np.sqrt(rmse)*1/744

  return (rmse<171)


#plt = model.plot(forecast)

###run all tests if TRUE

In [32]:
test_valid_model()

(True,
 "Make sure that you are using a model\ncovered in class in the variable 'model'.")

In [37]:
test_fitted_model()

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [34]:
test_valid_pred()

(True,
 'Make sure your prediction consists of integers\nor floating point numbers, and is a list or array of 744\nfuture predictions!')

In [38]:
test_accuracy_level_1()

np.True_

In [39]:
test_accuracy_level_2()

np.True_

In [40]:
test_accuracy_level_3()

np.True_